In [ ]:
import json
import openai
from openai import OpenAI
from google.colab import userdata
from PIL import Image
import requests
from pydantic import BaseModel
import base64
import io

In [ ]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
urlLimpa = "https://upload.wikimedia.org/wikipedia/commons/d/d8/Energy_assurance_only_a_microgrid_away_140113-M-OB827-007.jpg"
urlSuja = "https://www.aldo.com.br/blog/wp-content/uploads/2023/01/blog_aldosolar_Sujeira-no-modulo-pode-gerar-perda-de-receita.jpg"
urlSol = "https://elysia.com.br/wp-content/uploads/2021/05/sombreamento-paineis-solares-texto15.jpg"
urlNublado = "https://cdn.shopify.com/s/files/1/0596/3713/7578/files/Solar_panels_on_a_cloudy_day_480x480.webp?v=1728635656"

In [ ]:
def encode_image_to_base64(image_url):
    headers = {
    'User-Agent': 'College project demonstration (eduardo566460@fiap.com.br)'
}
    response = requests.get(image_url,headers=headers)

    response.raise_for_status()  # Raise an exception for bad status codes
    # Use Pillow to open the image and determine the format
    img = Image.open(io.BytesIO(response.content))
    img_format = img.format.lower()
    # Encode the image content to base64
    base64_image = base64.b64encode(response.content).decode('utf-8')
    # Return the data URL format
    return f"data:image/{img_format};base64,{base64_image}"
    urlLimpa = encode_image_to_base64(urlLimpa)

In [ ]:
class VerificaPlaca(BaseModel):
    id: int
    needCleaning: bool
    nivelSolar: int
    analise: str
urlLimpa64 = encode_image_to_base64(urlLimpa)
urlSuja64 = encode_image_to_base64(urlSuja)
urlSol64 = encode_image_to_base64(urlSol)
urlNublado64 = encode_image_to_base64(urlNublado)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
          "role": "system", "content": """
          You are an assistant that analyzes solar panel images.
          Verify with the image if the solar panel needs cleaning to make sure the efficiency will be high.
          If it needs cleaning, set 'needCleaning' to true.
          else set it to false.
          Also, measure the insolation in the image and classify it:
          - 3 for high insolation
          - 2 for medium
          - 1 for low
          Set the value in the `nivelSolar` column.
          also make a large analyse of the image and justify your response please
          Return the result as a JSON object in the format for better visualization:
          {
              "id": 1,
              "needCleaning": true/false,
              "nivelSolar": 1/2/3
              "analise": "your analyse"
          }
           {
              "id": 2,
              "needCleaning": true/false,
              "nivelSolar": 1/2/3
              "analise": "your analyse"
          }
        """
        },
        { "role": "user", "content":
         [
            {
                "type": "image_url",
                "image_url": {"url": urlLimpa}
            },
             {
                "type": "image_url",
                "image_url": {"url": urlSuja64}
            },
             {
                "type": "image_url",
                "image_url": {"url": urlSol64}
            },
             {
                "type": "image_url",
                "image_url": {"url": urlNublado64}
            },
        ]
        }
    ],
    response_format={"type": "json_object"}
)



painel = response.choices[0].message.content
print(painel)

{
    "solarPanels": [
        {
            "id": 1,
            "needCleaning": false,
            "nivelSolar": 3,
            "analise": "The solar panel appears clean and free of dust or debris, allowing for optimal sunlight absorption. It is inclined toward the sun and is situated in a clear area, which contributes to high insolation."
        },
        {
            "id": 2,
            "needCleaning": true,
            "nivelSolar": 1,
            "analise": "The solar panel shows significant dust accumulation, which can hinder performance and reduce efficiency. The insolation level is low due to the dirt covering much of the surface."
        },
        {
            "id": 3,
            "needCleaning": false,
            "nivelSolar": 2,
            "analise": "The solar panel appears mostly clean with some minor shadows but no significant dust or dirt. The position indicates moderate sun exposure, suggesting a medium level of insolation."
        },
        {
            "i